### The University of Melbourne, School of Computing and Information Systems
# COMP90086 Computer Vision, 2022 Semester 2

## Project

**Group Number:** `191`

**Student Name:**    `Junxian Wang, Xiaoru Yuan`

**Student ID:**     `1326418, 1314203`

In [2]:
import os
from glob import glob
import numpy as np
import cv2
import pandas as pd
from matplotlib import pyplot as plt
import time
import math

### 1. Set up

In [3]:
img_root = './Dataset/'
imgs_path_l = glob(os.path.join(img_root, '*-left.jpg'))
imgs_path_r = glob(os.path.join(img_root, '*-right.jpg'))
imgs_path_d = glob(os.path.join(img_root, '*-disparity.png'))
if not (len(imgs_path_l) == len(imgs_path_r) == len(imgs_path_d)):
    raise Exception('Path do not contain the same number of left, right and disparity images.')

imgs = {'left':[], 'right':[], 'disparity':[]}
for i in range(len(imgs_path_l)):
    imgs['left'].append(cv2.imread(imgs_path_l[i], cv2.IMREAD_GRAYSCALE))
    imgs['right'].append(cv2.imread(imgs_path_r[i], cv2.IMREAD_GRAYSCALE))
    imgs['disparity'].append(cv2.imread(imgs_path_d[i], cv2.IMREAD_GRAYSCALE))

imgs_df = pd.DataFrame(imgs)

### 2. Basic Stereo Matching

In [6]:
win_height = 21
win_width = 21

imgL = imgs_df['left'][0]
imgR = imgs_df['right'][0]

img_height = imgL.shape[0]
img_width = imgL.shape[1]

disparityBM = np.zeros((img_height, img_width))

for h in range(win_height//2, img_height-win_height//2):  # for each scanline
    print("curr h:", h)
    for w in range(win_width//2, img_width-win_width//2):    # for each pixel on scanline
        boxL = imgL[h-win_height//2:h+win_height//2, w-win_width//2:w+win_width//2]
        boxL_norm = np.linalg.norm(boxL)
        best_norm_corr = None
        for offset in range(0, w-win_width//2+1):
            boxR = imgR[h-win_height//2:h+win_height//2, w-offset-win_width//2:w-offset+win_width//2]
            # print(h,w,offset)
            # print(boxL.shape)
            # print(boxR.shape)

            norm_corr = np.sum(boxL * boxR)/(boxL_norm * np.linalg.norm(boxR))
            if best_norm_corr is None:
                best_norm_corr = norm_corr
            elif best_norm_corr < norm_corr:
                best_norm_corr = norm_corr
        disparityBM[h, w] = best_norm_corr
print(disparityBM)

curr h: 10
curr h: 11
curr h: 12
curr h: 13
curr h: 14
curr h: 15
curr h: 16
curr h: 17
curr h: 18
curr h: 19
curr h: 20
curr h: 21
curr h: 22
curr h: 23
curr h: 24
curr h: 25
curr h: 26
curr h: 27
curr h: 28
curr h: 29
curr h: 30
curr h: 31
curr h: 32
curr h: 33
curr h: 34
curr h: 35
curr h: 36
curr h: 37
curr h: 38
curr h: 39
curr h: 40
curr h: 41
curr h: 42
curr h: 43
curr h: 44
curr h: 45
curr h: 46
curr h: 47
curr h: 48
curr h: 49
curr h: 50
curr h: 51
curr h: 52
curr h: 53


KeyboardInterrupt: 

In [62]:
# # define parameters
# num_disp = 64 # try to tune the parameters to get better&smooth result
# block_size = 5
# disparityBM_ls = []

# elapsed_time = np.zeros(len(imgs_path_l))

# for i in range(len(imgs_path_l)):
#     start = time.time()

#     imgL = imgs_df['left'][i]
#     imgR = imgs_df['right'][i]
#     stereoBM = cv2.StereoBM_create(numDisparities=num_disp, blockSize=block_size)

#     # Compute the disparity image
#     disparityBM = stereoBM.compute(imgL,imgR)

#     # Scale the pixel values to between 0-255 (Or you can normalized to [0,1])
#     min = disparityBM.min()
#     max = disparityBM.max()
#     disparityBM = ((disparityBM - min) / (max - min)*255).astype(np.uint8)
#     disparityBM_ls.append(disparityBM)

#     # Compute runtime (in seconds per image)
#     end = time.time()
#     elapsed_time = end-start



### 3. Compare with ground truth

In [55]:
rms_ls = np.zeros(len(imgs_path_l))
fractions_4 = np.zeros(len(imgs_path_l))
fractions_2 = np.zeros(len(imgs_path_l))
fractions_1 = np.zeros(len(imgs_path_l))
fractions_05 = np.zeros(len(imgs_path_l))
fractions_025 = np.zeros(len(imgs_path_l))

for i in range(len(imgs_path_l)):
    # Load maps
    disparityBM = disparityBM_ls[i]
    ground_truth = imgs['disparity'][i]

    # Compute rms
    ms = np.square(np.subtract(disparityBM, ground_truth))
    rms = math.sqrt(ms/2)
    rms_ls[i] = rms

    # Compute
    fractions_4[i] = np.count_nonzero(rms < 4)
    fractions_2[i] = np.count_nonzero(rms < 2)
    fractions_1[i] = np.count_nonzero(rms < 1)
    fractions_05[i] = np.count_nonzero(rms < 0.5)
    fractions_025[i] = np.count_nonzero(rms < 0.25)